In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt
from mplfinance.original_flavor import candlestick_ohlc

In [ ]:
import requests

url = "https://www.nseindia.com/api/equity-stockIndices?index=NIFTY%20100"

headers = {
    "User-Agent": "Mozilla/5.0",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept": "application/json",
    "Referer": "https://www.nseindia.com/"
}

session = requests.Session()
session.get("https://www.nseindia.com", headers=headers)  # important

data = session.get(url, headers=headers).json()
print(data)  # should show 'data' among other keys
symbols = [item["symbol"] + ".NS" for item in data["data"]]

print(len(symbols))  # should be 100
print(symbols[:10])

In [ ]:
STOCK = ['RELIANCE.NS', 'TCS.NS', 'HDFCBANK.NS', ]
data = yf.download(STOCK, period='30d', interval='5m')
# save downloaded data
data.to_csv('data/multi_stock_data.csv')


In [ ]:
data.columns

In [ ]:
reliance_data = data.xs('RELIANCE.NS', level=1, axis=1)

In [ ]:
import talib as ta
reliance_data['MA20'] = ta.MA(reliance_data['Close'], timeperiod=20)

In [ ]:
reliance_data

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[
    go.Candlestick(
        x=reliance_data.index,
        open=reliance_data['Open'],
        high=reliance_data['High'],
        low=reliance_data['Low'],
        close=reliance_data['Close'],
        name='RELIANCE'
    )
])

fig.update_layout(
    title='RELIANCE.NS OHLC',
    xaxis_title='Date',
    yaxis_title='Price',
    xaxis_rangeslider_visible=True,  # 🔥 zoom slider
    # template='plotly_dark'            # optional
)

fig.show(renderer = 'browser')

In [ ]:
# read a parquet file and show it

In [ ]:
import pandas as pd

df = pd.read_parquet("../data/parquet/reliance.parquet")

In [ ]:
df

In [ ]:
import yfinance as yf

In [ ]:
rel_df = yf.download("RELIANCE.NS", period="2y", interval="1d")

In [ ]:
rel_df

In [ ]:
df = yf.download("RELIANCE.NS", period="2y", interval="1d", group_by="column")

# Flatten columns (THIS IS THE FIX)
df.columns = [col[0] if isinstance(col, tuple) else col for col in df.columns]

In [ ]:
df

In [ ]:
rel_df.reset_index()

In [ ]:
rel_df.columns

In [ ]:
import polars as pl
df_pl = pl.from_pandas(rel_df.reset_index())

In [ ]:
df_pl

In [ ]:
from src.data.fetchers.mutual_fund import fetch_nav

df = fetch_nav("120503")
print(df.tail())

In [ ]:
import polars as pl

In [ ]:
data = pl.read_parquet("../data/parquet/mf_registry.parquet")

In [ ]:
data

In [ ]:
import requests

URL = "https://www.advisorkhoj.com/mutual-funds-research/getPortfolioAnalysis"

HEADERS = {
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "X-Requested-With": "XMLHttpRequest",
    "Origin": "https://www.advisorkhoj.com",
    "Referer": "https://www.advisorkhoj.com/mutual-funds-research/"
}

def fetch_portfolio_by_slug(slug: str):
    body = f"scheme_amfi={slug}"

    resp = requests.post(
        URL,
        headers=HEADERS,
        data=body,
        timeout=20,
    )

    print("Status:", resp.status_code)
    print("Length:", len(resp.content))

    resp.raise_for_status()
    return resp.json()

data = fetch_portfolio_by_slug(
    "HDFC-Large-Cap-Fund-Growth-Option-Regular-Plan"
)
print(data.keys())


In [ ]:
data['schemePortfolioAnalysisResponse'].keys()

In [ ]:
data['schemePortfolioAnalysisResponse']['schemePortfolioList'], data['schemePortfolioAnalysisResponse']['sectorAllocationMap'], data['schemePortfolioAnalysisResponse']['assetAllocationMap']

In [ ]:
import requests

URL = "https://www.advisorkhoj.com/mutual-funds-research/autoSuggestAllMfSchemes"

HEADERS = {
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "X-Requested-With": "XMLHttpRequest",
    "Origin": "https://www.advisorkhoj.com",
    "Referer": "https://www.advisorkhoj.com/mutual-funds-research/"
}

def fetch_mfs(query: str):
    body = f"query={query}"

    resp = requests.post(
        URL,
        headers=HEADERS,
        data=body,
        timeout=20,
    )

    print("Status:", resp.status_code)
    print("Length:", len(resp.content))

    resp.raise_for_status()
    return resp.json()
fetch_mfs("a")

In [ ]:
import httpx
import polars as pl

BASE_URL = "https://api.mfapi.in/mf"

from urllib.parse import quote
from bs4 import BeautifulSoup
import re

BASE_OVERVIEW_URL = "https://www.advisorkhoj.com/mutual-funds-research/{scheme_name}"
NAV_URL = (
    "https://www.advisorkhoj.com/mutual-funds-research/"
    "getCompleteNavReportForFundOverview"
)
HEADERS = {
    "User-Agent": "Mozilla/5.0",
    "Accept": "text/html",
}

def fetch_fund_overview_html(scheme_name: str) -> str:
    """
    scheme_name example:
    'SBI ELSS Tax Saver FUND - REGULAR PLAN-GROWTH'
    """
    url = BASE_OVERVIEW_URL.format(
        scheme_name=quote(scheme_name)
    )
    # print(url)

    resp = httpx.get(url, headers=HEADERS, timeout=20)
    resp.raise_for_status()
    return resp.text

def extract_launch_date(html: str) -> str:
    """
    Returns date in DD-MM-YYYY format
    """
    soup = BeautifulSoup(html, "html.parser")
    # print(soup)

    for td in soup.select("table.sch_over_table td"):
        text = td.get_text(strip=True)
        if "Launch Date:" in text:
            # Example: "Launch Date: 31-03-1993"
            match = re.search(r"(\d{2}-\d{2}-\d{4})", text)
            if match:
                return match.group(1)

    raise ValueError("Launch Date not found")


def build_nav_params(scheme_name: str, launch_date: str) -> dict:
    """
    AdvisorKhoj expects scheme_amfi_name double-encoded
    """
    encoded = quote(quote(scheme_name.lower()))

    return {
        "scheme_amfi_name": encoded,
        "scheme_inception_date": launch_date,
    }
def fetch_nav_from_advisorkhoj(
    scheme_name: str,
) -> dict:
    """
    Full pipeline:
    HTML → launch date → NAV JSON
    """
    html = fetch_fund_overview_html(scheme_name)
    launch_date = extract_launch_date(html)

    params = build_nav_params(scheme_name, launch_date)

    resp = httpx.get(
        NAV_URL,
        headers={
            **HEADERS,
            "X-Requested-With": "XMLHttpRequest",
            "Referer": "https://www.advisorkhoj.com/mutual-funds-research/"
        },
        params=params,
        timeout=30,
    )

    resp.raise_for_status()
    return {
        "scheme": scheme_name,
        "launch_date": launch_date,
        "nav_data": resp.json(),
    }

scheme = "SBI ELSS Tax Saver FUND - REGULAR PLAN-GROWTH"
scheme = '-'.join([c for c in scheme.split(' ') if c != '-'])
data = fetch_nav_from_advisorkhoj(scheme)

print("Scheme:", data["scheme"])
print("Launch Date:", data["launch_date"])
print("NAV rows:", len(data["nav_data"]))
print("First entry:", data["nav_data"][0])

In [37]:
df = pl.read_parquet("../data/parquet/mf_holdings.parquet")

In [39]:
df['schemeName'].unique()

schemeName
str
"""Parag Parikh Flexi Cap Fund - …"
"""SBI ELSS Tax Saver Fund - Dire…"
"""SBI Large Cap Fund - Direct Pl…"
"""SBI Equity Hybrid Fund - Direc…"
"""UTI Nifty 50 Index Fund - Regu…"


In [83]:
import httpx
import string
import polars as pl
from itertools import product
from tqdm import tqdm
import re
import time


URL = "https://www.advisorkhoj.com/mutual-funds-research/autoSuggestAllMfSchemes"

HEADERS = {
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "X-Requested-With": "XMLHttpRequest",
    "Origin": "https://www.advisorkhoj.com",
    "Referer": "https://www.advisorkhoj.com/mutual-funds-research/",
}

MAX_RESULTS = 25
ALPHABET = string.ascii_lowercase
MAX_PREFIX_LEN = 6   # defensive


def slugify(name: str) -> str:
    name = name.lower()
    name = re.sub(r"[()]", " ", name)
    name = re.sub(r"[^a-z0-9]+", "-", name)
    return name.strip("-")


def fetch_all_advisorkhoj_schemes() -> pl.DataFrame:
    seen: set[str] = set()
    visited: set[str] = set()

    # Initial 2-letter prefixes
    stack = [
        a + b for a, b in product(ALPHABET, repeat=2)
    ]

    with httpx.Client(timeout=20, headers=HEADERS) as client:
        for prefix in tqdm(
            stack.copy(),
            desc="Crawling AdvisorKhoj schemes",
            unit="prefix",
            ncols=100,
        ):
            stack.append(prefix)  # seed stack for full DFS
            break
        stack = stack[:-1]  # clean seed hack

        while stack:
            prefix = stack.pop()

            if prefix in visited:
                continue
            visited.add(prefix)

            if len(prefix) > MAX_PREFIX_LEN:
                continue

            resp = client.post(URL, data={"query": prefix})
            resp.raise_for_status()
            results: list[str] = resp.json()

            before = len(seen)
            for name in results:
                if name:
                    seen.add(name.strip())
            after = len(seen)

            discovered_new = after > before
            capped = len(results) >= MAX_RESULTS

            # 🔑 recurse only if BOTH are true
            if capped and discovered_new:
                for c in ALPHABET:
                    stack.append(prefix + c)

            # be polite
            time.sleep(0.02)

    return pl.DataFrame(
        {
            "schemeName": sorted(seen),
            "schemeSlug": [slugify(x) for x in sorted(seen)],
            "source": ["advisorkhoj"] * len(seen),
        }
    )


In [84]:
df = fetch_all_advisorkhoj_schemes()

final_df = (
    df
    .with_columns([
        pl.lit("advisorkhoj").alias("source"),
        pl.col("schemeName").map_elements(slugify).alias("schemeSlug"),
    ])
    .unique(subset=["schemeName"])
    .sort("schemeName")
)

Crawling AdvisorKhoj schemes:   0%|                                     | 0/676 [00:00<?, ?prefix/s]

In [85]:
final_df.sort("schemeName")

schemeName,schemeSlug,source
str,str,str
"""360 ONE Dynamic Bond Fund - Re…","""360-one-dynamic-bond-fund-regu…","""advisorkhoj"""
"""360 ONE FLEXICAP FUND-REGULAR …","""360-one-flexicap-fund-regular-…","""advisorkhoj"""
"""360 ONE Focused Fund -Regular …","""360-one-focused-fund-regular-p…","""advisorkhoj"""
"""360 ONE LIQUID FUND REGULAR PL…","""360-one-liquid-fund-regular-pl…","""advisorkhoj"""
"""360 ONE Silver ETF""","""360-one-silver-etf""","""advisorkhoj"""
…,…,…
"""qsif Equity Long Short Fund - …","""qsif-equity-long-short-fund-gr…","""advisorkhoj"""
"""quant Aggressive Hybrid Fund -…","""quant-aggressive-hybrid-fund-g…","""advisorkhoj"""
"""quant ESG Integration Strategy…","""quant-esg-integration-strategy…","""advisorkhoj"""


In [86]:
final_df.write_parquet("../data/parquet/advisorkhoj_registry.parquet")

In [101]:
txn_df.columns

['symbol',
 'isin',
 'trade_date',
 'exchange',
 'segment',
 'series',
 'trade_type',
 'auction',
 'quantity',
 'price',
 'trade_id',
 'order_id',
 'order_execution_time']

In [106]:
import polars as pl
import plotly.express as px
import plotly.io as pio

pio.renderers.default = "browser"

txn_df = pl.read_csv("../data/user/tradebook-MF.csv")

txn_df = txn_df.with_columns(
    pl.col("trade_date").str.strptime(pl.Datetime, strict=False)
).filter(pl.col('symbol').is_in(['PARAG PARIKH FLEXI CAP FUND - DIRECT PLAN','PARAG PARIKH FLEXI CAP FUND - DIRECT PLAN GROWTH' ]))

pdf = txn_df.to_pandas()

fig = px.line(pdf, x="trade_date", y="quantity", color='symbol')
fig.show()


In [113]:
txn_df[0].to_pandas()

,symbol,isin,trade_date,exchange,segment,series,trade_type,auction,quantity,price,trade_id,order_id,order_execution_time
0,PARAG PARIKH FLEXI CAP FUND - DIRECT PLAN,INF879O01027,2024-05-21,BSE,MF,None,buy,False,127.238,78.5887,1171051501,1171051501,2024-05-21T00:00:00
